In [1]:
%matplotlib inline
import os, sys
from sklearn import linear_model, datasets
import pandas as pd
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

## 6.1.1 Create a classification model using a logistic regression!

* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
* Train a model using a testing dataset of 25%!

In [2]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df["target_name"]=iris['target_names'][iris_df['target']] 
iris_df.head()

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df["target_name"]=iris['target_names'][iris_df['target']] 
iris_df.head()

y = iris_df['target']
X = iris_df.drop(['target','target_name'],1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

model = linear_model.LogisticRegression(solver = 'lbfgs',multi_class = 'auto',max_iter=1000)
start = time.time()
model.fit(X_train, y_train)
runtime_sklearn = time.time()-start

pred = model.predict(X_train)
pred
#model.score(X, y)

array([1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2,
       1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1,
       0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2,
       2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0,
       2, 1, 1, 1, 2, 2, 1, 2, 0, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0, 0, 2, 1,
       2, 0])

* Visualize the model!

* Create a confusion matrix!

In [3]:
conf = confusion_matrix(y_test,model.predict(X_test))
conf

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]])

* Compute precision and recall for every class in your model!

In [4]:
accuracy_score(y_test,model.predict(X_test))

0.9736842105263158

In [5]:
# Initialize PySpark
import os, sys
os.environ["JAVA_HOME"]="/lrz/sys/compilers/java/jdk1.8.0_112"
APP_NAME = "PySpark Lecture"
SPARK_MASTER="local[16]"
import pyspark
import pyspark.sql
from pyspark.sql import Row
conf=pyspark.SparkConf()
conf=pyspark.SparkConf().setAppName(APP_NAME).set("spark.local.dir", os.path.join(os.getcwd(), "tmp"))
sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated...")

PySpark initiated...


Using Spark MLlib to create a Logistic Regression!

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression


iris_df
spark_df = spark.createDataFrame(iris_df)
#dataset = iris_df

#dataset = spark.createDataFrame(
#    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
 #   ["id", "hour", "mobile", "userFeatures", "clicked"])

assembler = VectorAssembler(
    inputCols=["sepal length (cm)", "sepal width (cm)", "petal length (cm)","petal width (cm)"],
    outputCol="target2")

output = assembler.transform(spark_df)
output.select("target2", "target").show(truncate=False)



+-----------------+------+
|target2          |target|
+-----------------+------+
|[5.1,3.5,1.4,0.2]|0     |
|[4.9,3.0,1.4,0.2]|0     |
|[4.7,3.2,1.3,0.2]|0     |
|[4.6,3.1,1.5,0.2]|0     |
|[5.0,3.6,1.4,0.2]|0     |
|[5.4,3.9,1.7,0.4]|0     |
|[4.6,3.4,1.4,0.3]|0     |
|[5.0,3.4,1.5,0.2]|0     |
|[4.4,2.9,1.4,0.2]|0     |
|[4.9,3.1,1.5,0.1]|0     |
|[5.4,3.7,1.5,0.2]|0     |
|[4.8,3.4,1.6,0.2]|0     |
|[4.8,3.0,1.4,0.1]|0     |
|[4.3,3.0,1.1,0.1]|0     |
|[5.8,4.0,1.2,0.2]|0     |
|[5.7,4.4,1.5,0.4]|0     |
|[5.4,3.9,1.3,0.4]|0     |
|[5.1,3.5,1.4,0.3]|0     |
|[5.7,3.8,1.7,0.3]|0     |
|[5.1,3.8,1.5,0.3]|0     |
+-----------------+------+
only showing top 20 rows



In [7]:
lr = LogisticRegression(featuresCol = 'target2', labelCol = 'target', maxIter=10)
start = time.time()
train = lr.fit(output)
runtime_spark = time.time() - start


* Measure the training time. Compare the training time with the scikit-learn implementation! Explain! 

In [8]:
print("Runtime spark: " + str(runtime_spark))
print("Runtime sklearn: " + str(runtime_sklearn))



Runtime spark: 2.914445161819458
Runtime sklearn: 0.0948183536529541


No Idea why spark is so much slower, maybe bc of introduced overhead for setting up spark?